In [3]:
# !pip install chromadb faiss-cpu

In [11]:
import urllib.request as request
from langchain.document_loaders import PyPDFLoader, PyMuPDFLoader, PDFPlumberLoader
from langchain.vectorstores import Chroma, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [3]:
request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch06/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf",
                    "data-files/pdf-example-01.pdf")

('data-files/pdf-example-01.pdf', <http.client.HTTPMessage at 0x1ad60a9d610>)

In [4]:
%%time
# pdf 파일의 텍스트 정보를 읽어서 페이지 단위로 반환
loader = PyPDFLoader('data-files/pdf-example-01.pdf')
pages = loader.load_and_split()

CPU times: total: 15.2 s
Wall time: 15.2 s


In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [8]:
documents = text_splitter.split_documents(pages)

In [9]:
len(documents)

497

In [13]:
# 분할된 문서들을 vectordb에 저장 (여기서는 Chroma)

# chroma_db = Chroma.from_documents(documents, OpenAIEmbeddings()) # 메모리에 저장
chroma_db = Chroma.from_documents(documents, OpenAIEmbeddings(), 
                                  persist_directory="vectordb/chroma.db") # 파일로 저장

In [14]:
chroma_db2 = Chroma(persist_directory="vectordb/chroma.db", 
                    embedding_function=OpenAIEmbeddings())

C:\Users\human\AppData\Local\Temp\ipykernel_13900\1059780125.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_db2 = Chroma(persist_directory="vectordb/chroma.db",


In [15]:
chroma_db._collection.count(), chroma_db2._collection.count()

(497, 497)

In [16]:
searched_documents = chroma_db2.similarity_search("북한의 교육과정")

In [23]:
searched_documents
print( searched_documents[3].page_content )

2023 북한인권보고서
184
데, 당국이 실시하는 반종교 교육을 통해 기독교를 접한 경우였다. 기
독교 관련 북한당국의 반종교 교육은 학교 교과과정에서 뿐만 아니
라 졸업 후 조직생활을 통해서도 이루어지고 있었다. 수집된 증언에 
따르면 북한에서 반종교 교육을 받고 종교에 대한 부정적 인식이 증
가했다고 한다. 기독교를 믿는 사람을 반동분자로 인식하고 있었다
는 증언들도 있었다. 한 증언자는 2015년에 계급교양관을 1달에 1번
씩 참관해야 했는데, 거기서 ‘종교는 침략자들이 북한에 가져온 것으
로 그들이 성경도 가져왔다’는 내용이 포함된 반종교 교육을 받았다
고 한다. 기독교인들은 제국주의적 침략의 앞잡이이므로 반민족적·
반혁명적 적대계층이라는 내용도 있었다고 한다. 다른 증언자는 교육
기관, 사회기관, 법기관에서 ‘종교는 허황된 것이고 거짓’이며 선교사
는 악한 자라고 세뇌가 될 정도로 지속적으로 교육하기 때문에 감히 
종교에 관심을 가질 생각도 하지 못했고 ‘선교사’라는 단어를 들으면 
지금도 무섭다고 한다. 인민반 강연 등에서 기독교를 믿는 사람은 반
동분자라고 하면서 이들을 보면 신고하라고 했다는 증언도 있었다. 
“주민들을 모아 놓고 교양하는 때가 있었는데, 그 때 처음 ‘성경책 
사진, 예배하는 모습의 사진’을 본 적이 있습니다. 반동조직이라고 
교육하면서 성경에는 미신에 관한 것이 적혀있어 이것을 읽게 되면 
사상이 변질되니 이런 책을 주변에서 보게 되면 신고하라고 했습니
다. 이들은 국가 반역자로 이 땅에서 사라져야 한다고 선전했습니
다. 그래서 재북시 저 뿐만이 아니라 북한 주민들은 성경책이 북한 
제도에 대해 안 좋게 적어놓은 책으로 알고 있었고 이를 소지하면 
죽임을 당한다고 알고 있었습니다. ”
북한에서 기독교를 탄압하는 이유는 기독교의 유일신 사상이 수
령 우상화 정책과 주체사상에 반하기 때문이라고 한다. 한 증언자는


In [24]:
faiss_db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [26]:
searched_documents2 = faiss_db.similarity_search("북한의 교육 과정")

In [32]:
print( searched_documents2[0].page_content )

2023 북한인권보고서
40
명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과
서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교
과서가 모든 학생에게 충분히 제공되지 않고 학년을 마치면 다음 학
년에 교과서를 물려주어야 했다는 사례가 다수 수집되었다. 소학교
부터 학교운영비, 꼬마계획 등의 비용을 내야했다는 진술이 꾸준히 
수집되고 있는데, 학교시설 현대화 작업이 진행되면서 학교꾸리기 
비용이 증가했다고 한다. 학교에서 요구하는 돈이나 물품은 교원에 
의해 사실상 강제되고 있었는데, 비용을 내지 못하는 경우 동급생들 
앞에서 망신을 주거나 비판하여 형편이 어려운 학생들은 학교를 그
만두는 선택을 하는 경우가 많다고 한다. 또한 도시와 농촌 간 교육
환경의 차이가 크며 대학입학에서 출신성분에 의한 차별이 있고, 교
육기회의 제공에도 경제력이 영향을 미치고 있어 성분·지역·경제
력에 따른 차별이 존재하는 것으로 나타났다. 교육환경도 열악한데, 
학교시설의 현대화 작업에도 불구하고 양호실, 도서관, 위생시설이 
없는 학교도 많은 것으로 보인다. 교원에 대한 경제적 보상도 적절
히 이루어지지 않아, 교원들은 생계를 유지하기 위해 잘사는 학부모
의 원조를 받거나 자신의 텃밭에 학생을 동원시키고 있어 학생들은 
제대로 된 교육여건을 보장받지 못하고 있는 것으로 나타났다. 또
한, 일반교육보다 정치사상교육을 앞세우고 있으며 교과과정에 실
탄사격을 하는 군사훈련을 편성하여 학생들을 의무적으로 참석하게 
하고 있다.
북한의 사회보장 제도로는 연로연금, 노동능력상실 연금, 유가족 
연금 등 생계가 결핍된 경우 기초적인 생계를 보장하기 위한 연금제
도가 있으며, 사회보험금의 성격을 지닌 보조금 제도가 있다. 연로


In [33]:
faiss_db.save_local('vectordb/faiss.db')

In [34]:
faiss_db2 = FAISS.load_local('vectordb/faiss.db',
                             OpenAIEmbeddings(),
                             allow_dangerous_deserialization=True)